In [5]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import gc


In [6]:
# Встановлюємо шляхи до файлів
csv_file_path = '../data/NSDUH_2015-2019.csv'
parquet_file_path = '../data/NSDUH_2015-2019.parquet'

# Тимчасова директорія для частин
temp_dir = '../data/temp_parquet_parts'
os.makedirs(temp_dir, exist_ok=True)

# Розмір частини (кількість рядків у чанку)
chunksize = 50_000


In [7]:
# Крок 1: Автоматичне визначення типів даних із допуском змішаних типів
print("Визначення типів даних...")
sample = pd.read_csv(csv_file_path, nrows=1000)  # Читаємо невеликий приклад
dtypes = {}
for col in sample.columns:
    col_dtype = sample[col].dtype
    if col_dtype == "float64" or col_dtype == "int64":
        dtypes[col] = "float32"  # Використовуємо float для змішаних числових типів
    elif col_dtype == "object":
        dtypes[col] = "category"  # Оптимізація для тексту


Визначення типів даних...


In [8]:
# Крок 2: Поетапне читання та конвертація
temp_files = []
try:
    for i, chunk in enumerate(pd.read_csv(csv_file_path, chunksize=chunksize, dtype=dtypes, low_memory=False)):
        # Перетворення в Arrow Table
        table = pa.Table.from_pandas(chunk, preserve_index=False)

        # Збереження частини у тимчасовий файл
        temp_file = os.path.join(temp_dir, f'part_{i}.parquet')
        pq.write_table(table, temp_file)
        temp_files.append(temp_file)

        # Очищення пам'яті
        del chunk, table
        gc.collect()

    # Об'єднання частин
    tables = [pq.read_table(temp_file) for temp_file in temp_files]
    combined_table = pa.concat_tables(tables)
    pq.write_table(combined_table, parquet_file_path)

    print(f"CSV файл успішно конвертовано у Parquet: {parquet_file_path}")

finally:
    # Видалення тимчасових файлів
    for temp_file in temp_files:
        os.remove(temp_file)
    os.rmdir(temp_dir)


CSV файл успішно конвертовано у Parquet: ../data/NSDUH_2015-2019.parquet
